In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=7fc00a94f005126353d1aa691758dda6b6f8fa134630dc171b72562030b4c8b2
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 

In [ ]:
# Imports

import cv2
import numpy as np
import pytesseract

In [ ]:
def sort_contours(cnts, method="left-to-right"):
	# initialize the reverse flag and sort index
	reverse = False
	i = 0
	# handle if we need to sort in reverse
	if method == "right-to-left" or method == "bottom-to-top":
		reverse = True
	# handle if we are sorting against the y-coordinate rather than
	# the x-coordinate of the bounding box
	if method == "top-to-bottom" or method == "bottom-to-top":
		i = 1
	# construct the list of bounding boxes and sort them from top to
	# bottom
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
		key=lambda b:b[1][i], reverse=reverse))
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)

In [ ]:
def box_extraction(img, cropped_dir_path):
  # Read the image
  img = cv2.imread("form.jpg", 0)

  # Thresholding the image
  (thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  # Invert the image
  img_bin = 255-img_bin
  cv2.imwrite("Image_bin.jpg",img_bin)

  # Defining a kernel length 
  kernel_length = np.array(img).shape[1]//80

  # A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
  verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))

  # A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
  hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))

  # A kernel of (3 X 3) ones.
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

  # Morphological operation to detect vertical lines from an image
  img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
  verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
  cv2.imwrite("verticle_lines.jpg",verticle_lines_img)

  # Morphological operation to detect horizontal lines from an image
  img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
  horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
  cv2.imwrite("horizontal_lines.jpg",horizontal_lines_img)

  # Weighting parameters, this will decide the quantity of an image to be added to make a new image.
  alpha = 0.5
  beta = 1.0 - alpha

  # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
  img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
  img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
  (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  cv2.imwrite("img_final_bin.jpg",img_final_bin)

  im2 = img.copy()
  contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  (contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

  idx = 0
  for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    # Draw a rect on copied image 
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Cropping the text blok for giving input to OCR
    cropped = im2[y:y + h, x:x + w]

    if (w > 80 and h > 20) and w > 3*h:
      idx += 1
      new_img = img[y:y+h, x:x+w]
      cv2.imwrite(cropped_dir_path+str(idx) + ".jpg", new_img)

box_extraction("form.jpg", "cropped")